In [1]:
%run fcpc.py

# CLT > quando houve aumento?

# Termos mais comuns em nomes de projetos

In [222]:
from collections import Counter
import re
from nltk.corpus import stopwords

In [223]:
ci = re.compile(r"[\-\s\.,º\d/]")

termos = [x for x in ci.split(" ".join(fcpc.PROJETO.unique().tolist()).lower()) if x and not ci.match(x)\
                                                                       and not x in stopwords.words("portuguese")]
c = Counter(termos)

c.most_common(10)

[('fcpc', 876),
 ('ufc', 506),
 ('ct', 385),
 ('cv', 347),
 ('bnb', 123),
 ('termo', 109),
 ('curso', 90),
 ('aditivo', 79),
 ('carta', 66),
 ('tc', 45)]

# Sazonalidade dos tipos de pagamento

# Distância entre nomes pela quantidade de projetos em comum

In [67]:
m_nome_proj_id = fcpc.groupby(["NOME", "PROJETO_ID"]).size().apply(lambda x: True).unstack().fillna(False)
m_nome_proj_id.head(2)

PROJETO_ID,0101,0104,1016,1017,1019,1021,1048,1088,1089,1109,...,3227,3230,3231,3232,3235,3236,3239,3253,3255,NÃO
NOME,,,,,,,,,,,,,,,,,,,,,
ABDENAGO ALVES DE BARROS,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABDIAS NEVES DE MELO NETO,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [68]:
from sklearn.neighbors import NearestNeighbors
# manhatan > quantidade de projetos da diferença simétrica
nn = NearestNeighbors(p=1).fit(m_nome_proj_id)

In [93]:
distances, neighbors = nn.kneighbors([m_nome_proj_id.loc["ABELARDO VIEIRA MOTA", :]], 20)

def pintae(v):
    onde_pintar = v["nome"].isin(["ABELARDO VIEIRA MOTA",
                        "ELIEZER TOME DE PAULA NETO",
                        "PAULO HENRIQUE MACEDO DE ARAUJO"])
    on = np.where(onde_pintar, "color: red", "")
    
    return pd.DataFrame(np.c_[on, on],
                        index=v.index,
                        columns=v.columns)
    
pd.DataFrame(list(zip(distances[0], m_nome_proj_id.index[neighbors[0]])), columns=["distancia", "nome"])\
.sort_values(["distancia", "nome"]).head(12)\
.style.apply(pintae, axis=None)

# Distância entre nomes pela quantidade de termos comuns

In [354]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [286]:
nomes = fcpc.NOME.unique()

In [391]:
tokenizer = re.compile(' ').split

cv = CountVectorizer(stop_words=stopwords.words("portuguese")).fit(nomes)
m = cv.transform(nomes)
nn = NearestNeighbors(metric="manhattan").fit(m)

In [400]:
def top_n(nome, n=3):
    
    ds, noms = nn.kneighbors(cv.transform([nome]), n_neighbors=n)
    
    return pd.DataFrame(list(zip(nomes[noms[0]], ds[0])), columns=["NOME", "DISTANCIA"])

In [401]:
top_n("ABELARDO VIEIRA MOTA")

,NOME,DISTANCIA
0,ABELARDO VIEIRA MOTA,"0,00"
1,MATILDE VIEIRA DOS SANTOS,"4,00"
2,JOAO VIEIRA CAVALCANTE,"4,00"


In [402]:
top_n("JOSE ANTONIO FERNANDES DE MACEDO")

,NOME,DISTANCIA
0,JOSE ANTONIO FERNANDES DE MACEDO,"0,00"
1,MARCOS ANTONIO DE MACEDO,"3,00"
2,HILDENIO JOSE MACEDO,"3,00"


In [403]:
top_n("JAVAM DE CASTRO MACHADO")

,NOME,DISTANCIA
0,JAVAM DE CASTRO MACHADO,"0,00"
1,ROSELIA C. DE CASTRO MACHADO,"2,00"
2,JONATHAN MACHADO DE CASTRO,"2,00"


In [404]:
tfidf = TfidfVectorizer(stop_words=stopwords.words("portuguese")).fit(nomes)
m = tfidf.transform(nomes)
nn = NearestNeighbors(metric="manhattan").fit(m)

In [405]:
top_n("ABELARDO VIEIRA MOTA")

,NOME,DISTANCIA
0,ABELARDO VIEIRA MOTA,"1,34"
1,ANTONIO ABELARDO COSTA SOUSA,"3,25"
2,JOAO VIEIRA DA SILVA,"3,40"


In [406]:
top_n("JOSE ANTONIO FERNANDES DE MACEDO")

,NOME,DISTANCIA
0,JOSE ANTONIO FERNANDES DE MACEDO,"2,02"
1,JOSE FERNANDES DA SILVA,"3,13"
2,ANTONIO JOSE DOS SANTOS,"3,37"


In [407]:
top_n("JAVAM DE CASTRO MACHADO")

,NOME,DISTANCIA
0,JAVAM DE CASTRO MACHADO,"1,34"
1,JONATHAN MACHADO DE CASTRO,"2,75"
2,ROSELIA C. DE CASTRO MACHADO,"2,85"
